Universidad del Valle de Guatemala

## Proyecto: Data Science

Andrea Amaya 19357 | Brandon Hernández	 19376 |		Martin Amado	19020 |   Juan Pablo Pineda 19087

In [ ]:
# run once
import glob, os
os.chdir('./cvs_data')

In [ ]:
import pandas as pd 
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None 

In [ ]:
data_frames = []
for file in glob.glob('*.csv'):
  print(file.split('.').pop(0))
  data_frames.append(pd.read_csv(file, encoding='utf-8'))